In [1]:
import qcodes
from qcodes.utils import installation_info as ii

ver = ii.get_qcodes_version()
print(25*'-')
print(f'QCoDeS version: {ver}')

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\wihpniel\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\wihpniel\.qcodes\logs\200110-12400-qcodes.log
-------------------------
QCoDeS version: 0.9.0rc1+56.g9648416db.dirty


In [2]:
import tempfile
import time

import numpy as np

from qcodes.dataset.measurements import Measurement
from qcodes.dataset.experiment_container import new_experiment
from qcodes.dataset.sqlite.database import initialise_or_create_database_at

In [3]:
def run_experiment(n, m, use_threads=False):
    """
    Run the experiment with 3*n^2 data points to be saved
    """

    with tempfile.TemporaryFile() as dbfile:
        dbfile.name += '.db'
        print(dbfile.name)
        initialise_or_create_database_at(dbfile.name)
        
        print(f'DB file: {dbfile.name}')
        
        new_experiment('profiling', 'no_sample')

        meas = (Measurement()
                .register_custom_parameter('X', paramtype='array')
                .register_custom_parameter('Y', paramtype='array')
                .register_custom_parameter('Z', paramtype='array', setpoints=['X', 'Y']))

        meas.write_period = 1.0
        t0 = time.perf_counter()

        saving_time = 0
        generation_time = 0
        
        with meas.run(use_threads=use_threads) as datasaver:
            init_time = time.perf_counter() - t0
            
            for _ in range(m):
                t1 = time.perf_counter()

                xdata, ydata = np.meshgrid(np.random.rand(n), np.random.rand(n))
                zdata = np.random.randn(*xdata.shape)
                time.sleep(0.5)
                
                t2 = time.perf_counter()
                
                generation_time += t2 - t1
                
                datasaver.add_result(('X', xdata), ('Y', ydata), ('Z', zdata))
                t3 = time.perf_counter()
                
                saving_time += t3 - t2
        t4 = time.perf_counter()
        saving_time += t4 - t3

    print('Report:')
    print(f'Number of data points saved: {3*n**2} {m} times')
    print(f'Init time:            {init_time} s')
    print(f'Data generation time: {generation_time} s')
    print(f'Data saving time:     {saving_time} s')
    print(f'Total time:           {t4-t0} s')

In [4]:
run_experiment(1000, 10, use_threads=False)

C:\Users\wihpniel\AppData\Local\Temp\tmpo5dtn0n9.db
Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]
Upgrading database; v1 -> v2: 100%|██████████| 1/1 [00:00<00:00, 220.96it/s]
Upgrading database; v2 -> v3: : 0it [00:00, ?it/s]
Upgrading database; v3 -> v4: : 0it [00:00, ?it/s]
Upgrading database; v4 -> v5: 100%|██████████| 1/1 [00:00<00:00, 125.03it/s]
Upgrading database; v5 -> v6: : 0it [00:00, ?it/s]
Upgrading database; v7 -> v8: 100%|██████████| 1/1 [00:00<00:00, 166.60it/s]
DB file: C:\Users\wihpniel\AppData\Local\Temp\tmpo5dtn0n9.db
Starting experimental run with id: 1. 
Report:
Number of data points saved: 3000000 10 times
Init time:            0.09915930000000017 s
Data generation time: 5.478591399999997 s
Data saving time:     4.0474167 s
Total time:           9.625191000000001 s


In [5]:
run_experiment(1000, 10, use_threads=True)

C:\Users\wihpniel\AppData\Local\Temp\tmp8w4w5azl.db
Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]
Upgrading database; v1 -> v2: 100%|██████████| 1/1 [00:00<00:00, 76.92it/s]
Upgrading database; v2 -> v3: : 0it [00:00, ?it/s]
Upgrading database; v3 -> v4: : 0it [00:00, ?it/s]
Upgrading database; v4 -> v5: 100%|██████████| 1/1 [00:00<00:00, 142.90it/s]
Upgrading database; v5 -> v6: : 0it [00:00, ?it/s]
Upgrading database; v7 -> v8: 100%|██████████| 1/1 [00:00<00:00, 201.52it/s]
DB file: C:\Users\wihpniel\AppData\Local\Temp\tmp8w4w5azl.db
Starting experimental run with id: 1. 
Report:
Number of data points saved: 3000000 10 times
Init time:            0.08605580000000046 s
Data generation time: 5.719024899999997 s
Data saving time:     0.9680663000000109 s
Total time:           6.7731612000000005 s
